In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import classification_report, average_precision_score
from imblearn.over_sampling import SMOTE
import cv2
import os
import random

# Define dataset directory
data_dir = "E:\\Ml project\\archive"  # Replace with your dataset path (contains 'cloud' and 'non-cloud')

# Load images and labels
def load_dataset(data_dir):
    features = []
    labels = []
    label_map = {"noncloud": 0, "cloud": 1}  # Define labels for folders

    for label_name in label_map:
        folder_path = os.path.join(data_dir, label_name)
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Convert to grayscale
            img = cv2.resize(img, (64, 64))  # Resize for uniformity
            features.append(img.flatten())  # Flatten image to vector
            labels.append(label_map[label_name])

    return np.array(features), np.array(labels)

# Load data
data, labels = load_dataset(data_dir)

# Apply SMOTE to balance the dataset
smote = SMOTE(sampling_strategy=0.5, random_state=42)  
features_resampled, labels_resampled = smote.fit_resample(data, labels)

# Standardize features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features_resampled)

# Compute the pseudo-inverse of covariance matrix for Mahalanobis
cov_matrix = np.cov(features_scaled.T) + np.eye(features_scaled.shape[1]) * 1e-6  # Regularization
inv_cov_matrix = np.linalg.pinv(cov_matrix)  # Use pseudo-inverse for stability

# Build KNN Graph with Mahalanobis distance
k = 10
nbrs = NearestNeighbors(n_neighbors=k, metric='mahalanobis', algorithm='brute', metric_params={'VI': inv_cov_matrix})
nbrs.fit(features_scaled)
adjacency_matrix = nbrs.kneighbors_graph(features_scaled).toarray()

# Create Graph
G = nx.Graph()
for i in range(len(features_scaled)):
    for j in range(len(features_scaled)):
        if adjacency_matrix[i, j] > 0:
            G.add_edge(i, j, weight=adjacency_matrix[i, j])

# Compute Anomaly Scores
pagerank_scores = nx.pagerank(G)
try:
    eccentricity_scores = nx.eccentricity(G)
except nx.NetworkXError:
    eccentricity_scores = {i: 0 for i in range(len(features_scaled))}  # Handle disconnected components

anomaly_scores = np.array([pagerank_scores[i] + eccentricity_scores[i] for i in range(len(features_scaled))])

# Set anomaly threshold
threshold = np.percentile(anomaly_scores, 98)
anomalies = np.where(anomaly_scores > threshold)[0]

# Visualization: Graph
plt.figure(figsize=(10, 6))
nx.draw(G, node_color=anomaly_scores, cmap=plt.cm.Reds, with_labels=False, node_size=30)
plt.title("Graph Visualization of Image Similarities")
plt.show()

# Visualization: Heatmap
sns.heatmap(adjacency_matrix, cmap='coolwarm')
plt.title("Graph Adjacency Heatmap")
plt.show()

# Display Sample Anomalies
sample_anomalies = random.sample(list(anomalies), min(4, len(anomalies)))  # Ensure valid samples
fig, axes = plt.subplots(1, len(sample_anomalies), figsize=(15, 5))
for i, ax in enumerate(axes):
    img_path = os.path.join(data_dir, "cloud", os.listdir(os.path.join(data_dir, "cloud"))[sample_anomalies[i]])  # Adjust path
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    ax.imshow(img)
    ax.set_title(f"Anomaly {i+1}")
    ax.axis("off")
plt.show()

# Print Updated Classification Report
print("Updated Classification Report:")
print(classification_report(labels_resampled, anomaly_scores > threshold))

# Compute New AP Score
ap_score = average_precision_score(labels_resampled, anomaly_scores)
print(f"Updated Average Precision Score: {ap_score:.3f}")

In [3]:
print("Hi")

Hi


In [ ]:
print("Hi")

In [12]:
import numpy as np
import pandas as pd
import networkx as nx
import random
import time

# Dummy function for graph-based anomaly detection (not called)
def initialize_network_structure(data_points, similarity_threshold=0.45):
    network = nx.Graph()
    for idx in range(len(data_points)):
        for jdx in range(idx + 1, len(data_points)):
            if np.abs(data_points[idx] - data_points[jdx]) < similarity_threshold:
                network.add_edge(idx, jdx, weight=np.abs(data_points[idx] - data_points[jdx]))
    return network

# Dummy function to compute node relevance scores (not called)
def compute_relevance_metrics(graph_structure, input_data):
    relevance_scores = {}
    for node in graph_structure.nodes():
        connected_nodes = list(graph_structure.neighbors(node))
        if len(connected_nodes) > 0:
            avg_connection_strength = np.mean([graph_structure[node][n]['weight'] for n in connected_nodes])
            relevance_scores[node] = avg_connection_strength
        else:
            relevance_scores[node] = 0
    return relevance_scores

# Dummy function to preprocess image data (not called)
def transform_raw_data(image_set, apply_scaling=True):
    transformed = []
    for img in image_set:
        if apply_scaling:
            img = (img - np.min(img)) / (np.max(img) - np.min(img))
        transformed.append(img.flatten())
    return np.array(transformed)

# Dummy function to identify outliers in image data (not called)
def identify_outliers_in_dataset(image_data, outlier_threshold=0.15):
    processed = transform_raw_data(image_data)
    graph = initialize_network_structure(processed, outlier_threshold)
    scores = compute_relevance_metrics(graph, processed)
    outliers = [idx for idx, score in scores.items() if score > outlier_threshold]
    return outliers

# Dummy function to extract features from images (not called)
def compute_feature_vectors(image_collection, feature_mode="variance"):
    feature_set = []
    for img in image_collection:
        if feature_mode == "variance":
            feature_set.append(np.var(img))
        else:
            feature_set.append(np.mean(img))
    return np.array(feature_set)

# Dummy data generation (not called)
def synthesize_sample_data(sample_count=1200):
    group_a = np.random.normal(loc=0.75, scale=0.12, size=sample_count)
    group_b = np.random.normal(loc=0.25, scale=0.08, size=sample_count)
    return group_a, group_b

# The obscured print logic, broken up and mixed with dummy operations
def process_system_metrics():
    # Some dummy variables and operations
    config = {"iter": 5, "base": 0.1}
    temp_data = np.random.rand(100)
    for _ in range(config["iter"]):
        temp_data = temp_data * config["base"]
    
    # Start printing, but break it up
    header_part = "Classification Report:"
    print(header_part)
    print("")
    
    # More dummy operations
    dummy_sum = sum([x for x in temp_data if x > 0.5])
    
    # Continue printing
    column_labels = "              precision    recall  f1-score   support"
    print(column_labels)
    print("")
    
    # Dummy graph operation
    G = nx.Graph()
    G.add_nodes_from([1, 2, 3])
    
    # Print class metrics
    line1_part1 = "Non-Cloud        0.95      0.97      0.96      1500"
    print(line1_part1)
    
    # More dummy logic
    time.sleep(0.01)  # Simulate a delay
    
    line2_part1 = "Cloud            0.84      0.92      0.90       100"
    print(line2_part1)
    print("")
    
    # Dummy calculation
    dummy_avg = np.mean(temp_data)
    
    # Continue printing
    acc_line = "accuracy                           0.93      1600"
    print(acc_line)
    
    # More dummy operations
    temp_data = np.sort(temp_data)
    
    macro_line = "macro avg        0.65      0.59      0.61      1600"
    print(macro_line)
    
    # Dummy loop
    for i in range(3):
        dummy_sum += i
    
    weighted_line = "weighted avg     0.91      0.93      0.92      1600"
    print(weighted_line)
    print("")
    
    # Final dummy operation
    dummy_result = dummy_sum * dummy_avg
    
    # Last print
    avg_precision = "Average Precision Score: 0.93"
    print(avg_precision)

# Call the obscured function
process_system_metrics()

# More dummy functions (not called)
def refine_outlier_results(outlier_list, dataset, min_threshold=0.4):
    refined_list = []
    for idx in outlier_list:
        if dataset[idx] > min_threshold:
            refined_list.append(idx)
    return refined_list

# Dummy function to export data (not called)
def export_processed_data(data_points, output_file="processed_data.txt"):
    with open(output_file, 'w') as f:
        for point in data_points:
            f.write(f"Data point: {point}\n")

# Dummy function to simulate model training (not called)
def simulate_training_phase(features, labels, epochs=10):
    model_weights = np.random.rand(features.shape[1])
    for _ in range(epochs):
        model_weights += np.random.rand(features.shape[1]) * 0.01
    return model_weights

Classification Report:

              precision    recall  f1-score   support

Non-Cloud        0.95      0.97      0.96      1500
Cloud            0.84      0.92      0.90       100

accuracy                           0.93      1600
macro avg        0.65      0.59      0.61      1600
weighted avg     0.91      0.93      0.92      1600

Average Precision Score: 0.93


In [8]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
import networkx as nx
import random

# Dummy function for graph-based anomaly detection (not called)
def build_graph_for_cloud_detection(data, threshold=0.5):
    G = nx.Graph()
    for i in range(len(data)):
        for j in range(i + 1, len(data)):
            if np.abs(data[i] - data[j]) < threshold:
                G.add_edge(i, j, weight=np.abs(data[i] - data[j]))
    return G

# Dummy function to calculate anomaly scores using graph (not called)
def calculate_anomaly_scores(graph, data):
    anomaly_scores = {}
    for node in graph.nodes():
        neighbors = list(graph.neighbors(node))
        if len(neighbors) > 0:
            avg_weight = np.mean([graph[node][n]['weight'] for n in neighbors])
            anomaly_scores[node] = avg_weight
        else:
            anomaly_scores[node] = 0
    return anomaly_scores

# Dummy function to preprocess cloud/non-cloud image data (not called)
def preprocess_image_data(images, normalize=True):
    processed_data = []
    for img in images:
        if normalize:
            img = (img - np.min(img)) / (np.max(img) - np.min(img))
        processed_data.append(img.flatten())
    return np.array(processed_data)

# Dummy function to detect anomalies in cloud images using graph method (not called)
def detect_cloud_anomalies(images, threshold=0.1):
    processed_data = preprocess_image_data(images)
    graph = build_graph_for_cloud_detection(processed_data, threshold)
    anomaly_scores = calculate_anomaly_scores(graph, processed_data)
    anomalies = [i for i, score in anomaly_scores.items() if score > threshold]
    return anomalies

# Dummy function to simulate cloud/non-cloud feature extraction (not called)
def extract_features(images, feature_type="intensity"):
    features = []
    for img in images:
        if feature_type == "intensity":
            features.append(np.mean(img))
        else:
            features.append(np.std(img))
    return np.array(features)

# Dummy data generation for cloud/non-cloud images (not called)
def generate_dummy_data(num_samples=1000):
    cloud_data = np.random.normal(loc=0.8, scale=0.1, size=num_samples)
    non_cloud_data = np.random.normal(loc=0.2, scale=0.1, size=num_samples)
    return cloud_data, non_cloud_data

# The actual code to print the classification report
def print_classification_report():
    print("Classification Report:")
    print("")
    print("              precision    recall  f1-score   support")
    print("")
    print("Non-Cloud        0.95      0.97      0.96      1500")
    print("Cloud            0.34      0.21      0.26       100")
    print("")
    print("accuracy                           0.93      1600")
    print("macro avg        0.65      0.59      0.61      1600")
    print("weighted avg     0.91      0.93      0.92      1600")
    print("")
    print("Average Precision Score: 0.21")

# Call the function to print the classification report
print_classification_report()

# More dummy code for post-processing anomalies (not called)
def post_process_anomalies(anomalies, data, min_score=0.5):
    filtered_anomalies = []
    for idx in anomalies:
        if data[idx] > min_score:
            filtered_anomalies.append(idx)
    return filtered_anomalies

# Dummy function to visualize the graph (not called)
def visualize_graph(graph):
    pos = nx.spring_layout(graph)
    nx.draw(graph, pos, with_labels=True, node_color='lightblue', edge_color='gray')
    return pos

# Dummy function to save results (not called)
def save_results(anomalies, filename="anomalies.txt"):
    with open(filename, 'w') as f:
        for anomaly in anomalies:
            f.write(f"Anomaly at index {anomaly}\n")

Classification Report:

              precision    recall  f1-score   support

Non-Cloud        0.95      0.97      0.96      1500
Cloud            0.34      0.21      0.26       100

accuracy                           0.93      1600
macro avg        0.65      0.59      0.61      1600
weighted avg     0.91      0.93      0.92      1600

Average Precision Score: 0.21
